In [26]:
#pip install opencv-python

In [27]:
#pip install torch

In [28]:
#pip install timm

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
from skimage.io import imread, imshow
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

import timm
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

2022-06-22 10:18:17.933312: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ameeth/Wclass/lib/python3.8/site-packages/cv2/../../lib64:/usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib:/usr/local/lib:
2022-06-22 10:18:17.933350: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
avail_pretrained_models = timm.list_models(pretrained=True)
avail_pretrained_models

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [3]:
os.getcwd()

'/home/ameeth/WC'

In [4]:
dir='/home/ameeth/WC/1125/Multi-class Weather Dataset'

In [5]:
scaler=transforms.Resize(size=(224,224))

In [61]:
#categories=['Cloud','Rain','Sunrise','Foggy']
#categories=['HAZE','RAINY','SUNNY','SNOWY']
#categories=['Cloudy','Foggy','Sunny','Snowy','Rainy']
categories=['Cloudy','Rain','Shine','Sunrise']

model=timm.create_model('densenet121',pretrained=True,global_pool='avg')
num_in_features=model.get_classifier().in_features
print(num_in_features)
#model.fc = nn.Identity()
model.classifier=nn.Identity()
#print(model.fc)
#print(model.get_classifier())
print(model.default_cfg)
#print(model.feature_info.module_name())
#print(model.feature_info.reduction())
#print(model.feature_info.channels())
#non,_=get_graph_node_names(model)
#print(non)
#features={'layer1.0.act2':'out'}
#feature_extractor=create_feature_extractor(model,return_nodes=features)
#print(feature_extractor)

1024
{'url': 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/densenet121_ra-50efcf5c.pth', 'num_classes': 1000, 'input_size': (3, 224, 224), 'pool_size': (7, 7), 'crop_pct': 0.875, 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'first_conv': 'features.conv0', 'classifier': 'classifier', 'architecture': 'densenet121'}


In [62]:
num_in_features=model.get_classifier()
num_in_features

Identity()

In [63]:
def get_vectors(image_name):
    img = Image.open(image_name) 
    img = img.convert("RGB")
    resized_image=scaler(img)
    image=torch.as_tensor(np.array(resized_image,dtype=np.float32)).transpose(2,0)[None]
    #feature_output=model.forward_features(image)
    #feature_output=feature_extractor(image)
    feature_output=model(image)
    
    return feature_output

In [64]:
count=0
print(count)

0


In [65]:
data=[]
for category in categories:
    path= os.path.join(dir, category)
    label=categories.index(category)
    
    for image in os.listdir(path):
        image_path=os.path.join(path,image)
        image_vector=get_vectors(image_path)
        
        #print(image_vector.shape)
        image_list_vector=image_vector.tolist()
        image_numpy_vector=np.array(image_list_vector,dtype=object)
        #print(type(image_numpy_vector),image_numpy_vector.shape)
       
        #r1=image_numpy_vector.shape[1]
        #r2=image_numpy_vector.shape[2]
        #r3=image_numpy_vector.shape[3]
        #print(r1,r2,r3)
        
        image_new=np.reshape(image_numpy_vector,(1024));
        print(image_new.shape)
        
        image_new1=image_new.astype('float')
        #m = 49
        #a = np.array([2, 3, 4, 8, 9, 10])
        #b = image_new.reshape(-1, m).mean(axis=1)
        #b=b.astype('float')
        data.append([image_new1,label])
        #data.append([b,label])
        count=count+1
        #print(count,end=' ')
    break



(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)
(1024,)


KeyboardInterrupt: 

In [103]:
print(len(data))

1125


In [104]:
pickle.dump(data,open('resnet18test5_1125.pickle','wb'))

In [ ]:
model=timm.create_model('resnet101',pretrained=True,num_classes=10,global_pool='avg')
print(model.default_cfg)

In [ ]:
num_in_features=model.get_classifier().in_features
num_in_features

In [ ]:
count=0
print(count)

In [67]:
data=[]
for category in categories:
    path= os.path.join(dir, category)
    label=categories.index(category)
    
    for image in os.listdir(path):
        image_path=os.path.join(path,image)
        image_vector=get_vectors(image_path)
        #print(type(image_vector))
        image_list_vector=image_vector.tolist()
        image_numpy_vector=np.array(image_list_vector,dtype=object)
        #print(type(image_numpy_vector),image_numpy_vector.shape)
        #r1=image_numpy_vector.shape[1]
        #r2=image_numpy_vector.shape[2]
        #r3=image_numpy_vector.shape[3]
        #print(r1,r2,r3)
        #image_new=np.flatten(image_numpy_vector);
        image_new1=image_numpy_vector.flatten()
        #image_new1=image_new.astype('float')
        print(image_new1.shape)
        #data.append([image_new1,label])
        count=count+1
        #print(count,end=' ')
    break

(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)
(25088,)


KeyboardInterrupt: 

In [ ]:
pickle.dump(data,open('resnet101_1125.pickle','wb'))

In [21]:
model=timm.create_model('resnet152',pretrained=True,num_classes=10,global_pool='avg')
print(model.default_cfg)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet152_a1h-dc400468.pth" to /home/ameeth/.cache/torch/hub/checkpoints/resnet152_a1h-dc400468.pth


{'url': 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet152_a1h-dc400468.pth', 'num_classes': 1000, 'input_size': (3, 224, 224), 'pool_size': (7, 7), 'crop_pct': 0.95, 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225), 'first_conv': 'conv1', 'classifier': 'fc', 'architecture': 'resnet152'}


In [22]:
num_in_features=model.get_classifier().in_features
num_in_features

2048

In [23]:
count=0
print(count)

0


In [24]:
data=[]
for category in categories:
    path= os.path.join(dir, category)
    label=categories.index(category)
    
    for image in os.listdir(path):
        image_path=os.path.join(path,image)
        image_vector=get_vectors(image_path)
        #print(type(image_vector))
        image_list_vector=image_vector.tolist()
        image_numpy_vector=np.array(image_list_vector,dtype=object)
        #print(type(image_numpy_vector),image_numpy_vector.shape)
        r1=image_numpy_vector.shape[1]
        r2=image_numpy_vector.shape[2]
        r3=image_numpy_vector.shape[3]
        #print(r1,r2,r3)
        image_new=np.reshape(image_numpy_vector,(r1*r2*r3));
        image_new1=image_new.astype('float')
        #print(image_new1.shape)
        data.append([image_new1,label])
        count=count+1
        print(count,end=' ')
    

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [25]:
pickle.dump(data,open('resnet152_1125.pickle','wb'))